# Filtering PDB for RNA-RNA interactions
### Goals:
1. Download RNA from the PDB
2. Filter for those that interact with other RNAs
3. Determine interface of interaction on all interacting molecule

### Outcomes
With the RNA molecule interaction interfaces, the structures can be fed into veRNAl to mine motifs. Finding motifs in the interaction site will like give a good positive validation set

In [15]:
import numpy as np
from Bio.PDB import *
import nglview as nv

# Parse Structure File
# first do for one then try to do for many
parser = MMCIFParser()
structure = parser.get_structure('6zmi', '../data/rna_structure_representative_set/6zmi.cif')

C:\Users\jonab\anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L5 is discontinuous at line 224858.
  warnings.warn(
C:\Users\jonab\anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L7 is discontinuous at line 225070.
  warnings.warn(
C:\Users\jonab\anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L8 is discontinuous at line 225073.
  warnings.warn(
C:\Users\jonab\anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain LA is discontinuous at line 225077.
  warnings.warn(
C:\Users\jonab\anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain LI is discontinuous at line 225078.
  warnings.warn(
C:\Users\jonab\anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain LP is discontinuous at line 225079.
  warnings.warn

In [16]:
view = nv.show_biopython(structure)
view.clear_representations()
#add ribbons
view.add_licorice('rna', color='red')
view.add_cartoon('protein', color='blue')
#add ball and stick for non-rotien
# view.add_ball_and_stick('not protein')
view

TypeError: %c requires int or char

In [17]:
for model in structure:
    print(f'model: {model}')

model: <Model id=0>


In [18]:
structure[0].get_list()

[<Chain id=L5>,
 <Chain id=L7>,
 <Chain id=L8>,
 <Chain id=LA>,
 <Chain id=LB>,
 <Chain id=LC>,
 <Chain id=LD>,
 <Chain id=LE>,
 <Chain id=LF>,
 <Chain id=LG>,
 <Chain id=LH>,
 <Chain id=LI>,
 <Chain id=LJ>,
 <Chain id=LL>,
 <Chain id=LM>,
 <Chain id=LN>,
 <Chain id=LO>,
 <Chain id=LP>,
 <Chain id=LQ>,
 <Chain id=LR>,
 <Chain id=LS>,
 <Chain id=LT>,
 <Chain id=LU>,
 <Chain id=LV>,
 <Chain id=LW>,
 <Chain id=LX>,
 <Chain id=LY>,
 <Chain id=LZ>,
 <Chain id=La>,
 <Chain id=Lb>,
 <Chain id=Lc>,
 <Chain id=Ld>,
 <Chain id=Le>,
 <Chain id=Lf>,
 <Chain id=Lg>,
 <Chain id=Lh>,
 <Chain id=Li>,
 <Chain id=Lj>,
 <Chain id=Lk>,
 <Chain id=Ll>,
 <Chain id=Lm>,
 <Chain id=Ln>,
 <Chain id=Lo>,
 <Chain id=Lp>,
 <Chain id=Lr>,
 <Chain id=Ls>,
 <Chain id=Lt>,
 <Chain id=Lz>,
 <Chain id=S2>,
 <Chain id=SA>,
 <Chain id=SB>,
 <Chain id=SD>,
 <Chain id=SE>,
 <Chain id=SF>,
 <Chain id=SH>,
 <Chain id=SI>,
 <Chain id=SK>,
 <Chain id=SL>,
 <Chain id=SP>,
 <Chain id=SQ>,
 <Chain id=SR>,
 <Chain id=SS>,
 <Chain 

Two RNA chains: Q,R
Four Protein chains: A,B,C,D

In [19]:
struc_dict = MMCIF2Dict.MMCIF2Dict('../data/rna_structure_representative_set/1a9n.cif')
for key in struc_dict.keys():
        print(key)

data_
_entry.id
_audit_conform.dict_name
_audit_conform.dict_version
_audit_conform.dict_location
_database_2.database_id
_database_2.database_code
_pdbx_database_status.status_code
_pdbx_database_status.entry_id
_pdbx_database_status.recvd_initial_deposition_date
_pdbx_database_status.deposit_site
_pdbx_database_status.process_site
_pdbx_database_status.status_code_sf
_pdbx_database_status.status_code_mr
_pdbx_database_status.SG_entry
_pdbx_database_status.pdb_format_compatible
_pdbx_database_status.status_code_cs
_audit_author.name
_audit_author.pdbx_ordinal
_citation.id
_citation.title
_citation.journal_abbrev
_citation.journal_volume
_citation.page_first
_citation.page_last
_citation.year
_citation.journal_id_ASTM
_citation.country
_citation.journal_id_ISSN
_citation.journal_id_CSD
_citation.book_publisher
_citation.pdbx_database_id_PubMed
_citation.pdbx_database_id_DOI
_citation_author.citation_id
_citation_author.name
_citation_author.ordinal
_cell.entry_id
_cell.length_a
_cell.l

In [20]:
for res in structure[0].get_residues():
        print(res.get_parent().get_id())
        break

L5


In [21]:
# make a list of protein residues and RNA residues
protein_residues = []
rna_residues = []

for res in structure[0].get_residues():
    # skip non-polymer and water residues
    if res.id[0].startswith("H") or res.id[0].startswith("W"):
        continue
    if is_aa(res):
        protein_residues.append(res)
    else:
        rna_residues.append(res)

In [22]:
if rna_residues[0] == rna_residues[1]:
    print(t)
else:
    print("f")

f


### RNA-RNA interactions

##### Notes:
Hydrogen bond distance = 1.5-2.5 angstroms

N-H bond distance = 1 Angstrom

In [ ]:
# Find binding residues between RNA and protein
# take the distances between alpha_carbons of all protein and RNA residues
cutoff = 4

# binding_residues = dict()

# dictionary of rna protein-binding-residues and protein rna-binding-residues 
rna_rbr = dict()

for RNA_res_1 in rna_residues:
    for RNA_res_2 in rna_residues:
        # skip residues that are the same, ones already done and residues in the same chain
        if RNA_res_1 == RNA_res_2:
            continue
        if RNA_res_1 in rna_rbr.values() and RNA_res_2 in rna_rbr.values():
            continue
        if RNA_res_1.get_parent().get_id() == RNA_res_2.get_parent().get_id():
            continue
        distances = []
        for atom_1 in RNA_res_1:
            for atom_2 in RNA_res_2:
                #subtract the two position vectors
                diff_vector = atom_1.coord - atom_2.coord
                #to get a positive value we square the difference vector
                #we then take the square root to go back to the original scale
                distances.append(np.sqrt(np.sum(diff_vector * diff_vector)))
                # we get the nearest atom using min(distances) and see if it falls inside
                # the cutoff
        if min(distances) < cutoff:
            # append RNA residue_1 to rna_rbr (rna-rna binding residues)
            RNA_chain_2 = RNA_res_2.get_parent().get_id()
            if RNA_chain_2 in rna_rbr.keys():
                if RNA_res_1 not in rna_rbr[RNA_chain_2]: rna_rbr[RNA_chain_2].append(RNA_res_1)
            else:
                rna_rbr[RNA_chain_2] = [RNA_res_1]
            # Append the other residue 2
            RNA_chain_1 = RNA_res_1.get_parent().get_id()
            if RNA_chain_1 in rna_rbr.keys():
                if RNA_res_2 not in rna_rbr[RNA_chain_1]: rna_rbr[RNA_chain_1].append(RNA_res_2)
            else:
                rna_rbr[RNA_chain_1] = [RNA_res_2]
            

### Protein-RNA interactions

In [107]:

# Find binding residues between RNA and protein
# take the distances between alpha_carbons of all protein and RNA residues
cutoff = 5

# binding_residues = dict()

# dictionary of rna protein-binding-residues and protein rna-binding-residues 
rna_pbr = dict()
protein_rbr = dict()

for RNA_res in rna_residues:
    for prot_res in protein_residues:
        try:
            prot_alpha_carbon = prot_res['CA']
        except:
            print(f'error protein residue alpha carbon not found id: {prot_res.id}')
            continue
        distances = []
        for atom in RNA_res:
            #subtract the two position vectors
            diff_vector = prot_alpha_carbon.coord - atom.coord
            #to get a positive value we square the difference vector
            #we then take the square root to go back to the original scale
            distances.append(np.sqrt(np.sum(diff_vector * diff_vector)))
            # we get the nearest atom using min(distances) and see if it falls inside
            # the cutoff
        if min(distances) < cutoff:
            # append RNA residue to rna_pbr (rna protein binding residues)
            protein_chain = prot_res.get_parent().get_id()
            if protein_chain in rna_pbr.keys():
                if RNA_res not in rna_pbr[protein_chain]: rna_pbr[protein_chain].append(RNA_res)
            else:
                rna_pbr[protein_chain] = [RNA_res]
            # Append AA residue to protein_rbr
            rna_chain = RNA_res.get_parent().get_id()
            if rna_chain in protein_rbr.keys():
                if prot_res not in protein_rbr[rna_chain]: protein_rbr[rna_chain].append(prot_res)
            else:
                protein_rbr[rna_chain] = [prot_res]
            

error protein residue alpha carbon not found id: (' ', 193, ' ')
error protein residue alpha carbon not found id: (' ', 193, ' ')
error protein residue alpha carbon not found id: (' ', 193, ' ')


In [108]:
rna_pbr

{'A': [<Residue G het=  resseq=2 icode= >, <Residue A het=  resseq=3 icode= >]}

In [109]:
protein_rbr

{'R': [<Residue PHE het=  resseq=125 icode= >,
  <Residue GLY het=  resseq=126 icode= >,
  <Residue ARG het=  resseq=134 icode= >,
  <Residue HIS het=  resseq=124 icode= >,
  <Residue LYS het=  resseq=127 icode= >,
  <Residue ALA het=  resseq=128 icode= >]}

In [110]:
# view = nv.demo()

view = nv.show_biopython(structure[0])
view.clear_representations()
view.add_licorice('rna')#, color='red')
# view.add_cartoon('protein', color='black')
# use hex values for now.
residues = structure[0]['R'].get_residues()
#this is a bit of a hack to set the binding residues to red in the visualization
colors = ['0x0000FF' if r not in rna_pbr['A'] else '0xFF0000' for r in residues]
# view.add_licorice('rna')
# view.add_cartoon('protein')
view._set_color_by_residue(colors, component_index=0, repr_index=0)
view

NGLWidget()